# Práctica 3. Reconocimiento de Entidades Nombradas. NER

Andrés González Flores

Procesamiento de Lenguaje Natural

Facultad de Ingeniería, UNAM

## Objetivo

Realizar un reconocimiento de Entidades Nombradas (NER) a partir de un modelo secuencial y con el corpus ‘ner dataset.csv’ que se proporciona.

## Instrucciones

Se deberán seguir los siguientes pasos:

1. Obtener las sentencias a partir del csv. En este archivo, se indica cada inicio de sentencia con ‘Sentence: n’. Se cuenta con 1000 sentencias que conformarán el corpus de entrenamiento y evaluación.
2. Preprocesar los datos.
3. Separar los datos en corpus de entrenamiento (70%) y corpus de evaluación (30%).
4. Entrenar un modelo secuencial a partir del corpus de entrenamiento. Deberán definirse los hiperparámetros.
5. Evaluar el desempeño del sistema a partir del corpus de evaluación y con la métrica de Exactitud (Accuracy).
6. Ejemplificar el reconocimiento de entidades nombradas con 5 sentencias del corpus de evaluación.

## Dessarrollo

In [2]:
import csv
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import operator
from itertools import chain
from tqdm.notebook import tqdm
from pprint import pprint

In [3]:
# Definición de constantes
SEED = 42
CORPUS_PATH = './ner_dataset.csv'
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<unk>'

np.random.seed(SEED)

### Paso 1. Obtener sentencias.


In [4]:
with open(CORPUS_PATH, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    csvlist = list(reader)

In [5]:
sentences = []
sent_i = []
for row in csvlist[1:]:
    if row[0] is not '':
        sentences.append(sent_i)
        sent_i = []
    sent_i.append((row[1], row[2]))
# Mi algoritmo está feo: agrega la primer sentencia vacía y no agrega la última
del sentences[0]
sentences.append(sent_i)

In [6]:
print(f'Número de oraciones = {len(sentences)}\n')
print('Ejemplos:')
print('\n'.join([' '.join([w for w, tag in sent]) for sent in sentences[:5] ]))

Número de oraciones = 1000

Ejemplos:
Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "
They marched from the Houses of Parliament to a rally in Hyde Park .
Police put the number of marchers at 10000 while organizers claimed it was 1,00,000 .
The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton .


### Paso 2. Preprocesamiento

In [7]:
freq_tokens = Counter([w for w,_ in chain(*sentences)])
freq_tokens.most_common(10)

[('the', 1111),
 ('.', 995),
 (',', 637),
 ('in', 576),
 ('of', 568),
 ('to', 505),
 ('a', 449),
 ('and', 391),
 ('The', 241),
 ("'s", 202)]

Agrego el diccionario de entrada

In [8]:
word_to_index = { 
    x[0] : i for i, x in enumerate(freq_tokens.most_common())
    if freq_tokens[x[0]] > 1
}

pprint(list(word_to_index.items())[:10])

[('the', 0),
 ('.', 1),
 (',', 2),
 ('in', 3),
 ('of', 4),
 ('to', 5),
 ('a', 6),
 ('and', 7),
 ('The', 8),
 ("'s", 9)]


Agrego los identificadores UNK, BOS Y EOS al vocabulario de entrada

In [9]:
word_to_index[UNK] = max(word_to_index.values())+1
word_to_index[BOS] = word_to_index[UNK]+1
word_to_index[EOS] = word_to_index[BOS]+1

pprint(list(word_to_index.items())[-10:])

[('collapse', 2102),
 ('Barno', 2103),
 ('reunification', 2104),
 ('Berlin', 2105),
 ('Kohl', 2106),
 ('proud', 2107),
 ('wall', 2108),
 ('<unk>', 2109),
 ('<BOS>', 2110),
 ('<EOS>', 2111)]


Agrego el vocabulario de salida (las etiquetas)

In [10]:
freq_tags = Counter([tag for _,tag in chain(*sentences)])
freq_tags.most_common()

[('', 18843),
 ('B-ge', 583),
 ('B-gpe', 543),
 ('B-rg', 413),
 ('I-per', 400),
 ('B-tim', 358),
 ('B-per', 327),
 ('I-rg', 290),
 ('I-ge', 105),
 ('I-tim', 77),
 ('B-art', 39),
 ('I-gpe', 26),
 ('I-art', 22),
 ('B-eve', 20),
 ('I-eve', 16),
 ('B-nat', 9),
 ('I-nat', 5)]

In [11]:
tag_to_index = {
    y[0] : i for i, y in enumerate(freq_tags.most_common())
}
pprint(tag_to_index)

{'': 0,
 'B-art': 10,
 'B-eve': 13,
 'B-ge': 1,
 'B-gpe': 2,
 'B-nat': 15,
 'B-per': 6,
 'B-rg': 3,
 'B-tim': 5,
 'I-art': 12,
 'I-eve': 14,
 'I-ge': 8,
 'I-gpe': 11,
 'I-nat': 16,
 'I-per': 4,
 'I-rg': 7,
 'I-tim': 9}


Creo los vocabularios inversos

In [12]:
index_to_word = [w for w in word_to_index.keys()]
print(f'Tamaño del vocabulario: {len(index_to_word)}')
print('Ejemplos de palabras:')
print(', '.join(index_to_word[::350]))
print()

index_to_tag = [tag for tag in tag_to_index.keys()]
print('Vocabulario de etiquetas:')
pprint(', '.join(index_to_tag))

Tamaño del vocabulario: 2112
Ejemplos de palabras:
the, parliament, measure, claim, flown, burning, Tokar

Vocabulario de etiquetas:
(', B-ge, B-gpe, B-rg, I-per, B-tim, B-per, I-rg, I-ge, I-tim, B-art, I-gpe, '
 'I-art, B-eve, I-eve, B-nat, I-nat')


Creo un método que devuelve siempre un indice del vocabulario o el identificador UNK

In [13]:
def word_safe_vocab_index(w, vocab, ix_UNK):
    try:
        return vocab[w]
    except KeyError:
        return ix_UNK

Armo el corpus de entrenamiento

In [14]:
ix_UNK = word_to_index[UNK]
ix_BOS = word_to_index[BOS]
ix_EOS = word_to_index[EOS]

BOS_append = [(ix_BOS, tag_to_index[''])] # BOS y su etiqueta
EOS_append = [(ix_EOS, tag_to_index[''])] # EOS y su etiqueta

corpus = [
    BOS_append + [ # Agrego BOS y su etiqueta
        # Tupla con el valor del índice de la palabra y el índice de la etiqueta
        (
            word_safe_vocab_index(w, word_to_index, ix_UNK),
            tag_to_index[tag]
        ) # Fin tupla
        for w, tag in sent # Para cada palabra en la sentencia
    ] + EOS_append # Fin append
    for sent in sentences # Por cada sentencia en la lista de sentencias
]
# pprint(corpus[:5])
print('\n\n'.join([' '.join([f'({ix_w}, {ix_tag})' for ix_w, ix_tag in sent]) for sent in corpus[:5] ]))

(2110, 0) (904, 0) (4, 0) (905, 0) (14, 0) (906, 0) (243, 0) (244, 1) (5, 0) (298, 0) (0, 0) (127, 0) (3, 0) (53, 1) (7, 0) (1282, 0) (0, 0) (679, 0) (4, 0) (99, 2) (77, 0) (17, 0) (16, 0) (71, 0) (1, 0) (2111, 0)

(2110, 0) (2109, 0) (4, 0) (177, 0) (31, 0) (3, 0) (0, 0) (680, 0) (551, 0) (0, 0) (458, 0) (59, 0) (459, 0) (2109, 0) (18, 0) (245, 0) (1283, 0) (25, 0) (28, 0) (200, 6) (2109, 0) (460, 0) (2109, 0) (28, 0) (7, 0) (28, 0) (2109, 0) (0, 0) (2109, 0) (1, 0) (28, 0) (2111, 0)

(2110, 0) (134, 0) (906, 0) (17, 0) (0, 0) (2109, 0) (4, 0) (2109, 0) (5, 0) (6, 0) (1284, 0) (3, 0) (2109, 1) (2109, 8) (1, 0) (2111, 0)

(2110, 0) (268, 0) (340, 0) (0, 0) (341, 0) (4, 0) (1285, 0) (22, 0) (1286, 0) (135, 0) (1287, 0) (299, 0) (32, 0) (19, 0) (2109, 0) (1, 0) (2111, 0)

(2110, 0) (8, 0) (298, 0) (552, 0) (15, 0) (0, 0) (1288, 0) (4, 0) (0, 0) (387, 0) (553, 0) (4, 0) (554, 1) (9, 0) (907, 0) (461, 3) (462, 7) (3, 0) (0, 0) (128, 0) (2109, 2) (2109, 0) (2109, 0) (4, 0) (2109, 1) (1, 0) 

### Paso 3. Separar corpus de entrenamiento y evaluacion

In [15]:
train_corpus, val_corpus = train_test_split(corpus, test_size=0.3, random_state=SEED)
print("Ejemlos del corpus de entrenamiento")
for sent in train_corpus[:5]:
    print(sent)
print()
print("Ejemlos del corpus de validación")
for sent in val_corpus[:5]:
    print(sent)

Ejemlos del corpus de entrenamiento
[(2110, 0), (123, 0), (13, 0), (0, 0), (692, 0), (341, 0), (4, 0), (178, 0), (1599, 0), (636, 0), (1211, 0), (11, 0), (326, 0), (480, 0), (5, 0), (2109, 0), (129, 0), (2, 0), (171, 0), (1670, 0), (1, 0), (2111, 0)]
[(2110, 0), (8, 0), (487, 0), (19, 0), (1776, 0), (27, 0), (0, 0), (131, 0), (668, 0), (3, 0), (2109, 1), (2, 0), (7, 0), (0, 0), (2109, 0), (4, 0), (50, 6), (667, 4), (2, 0), (6, 0), (2109, 0), (4, 0), (2109, 2), (49, 6), (2109, 4), (2109, 4), (2, 0), (19, 0), (13, 0), (5, 0), (565, 0), (1777, 0), (5, 0), (340, 0), (46, 0), (1778, 0), (15, 0), (2109, 2), (1, 0), (2111, 0)]
[(2110, 0), (443, 0), (51, 0), (56, 0), (2, 0), (606, 2), (144, 6), (93, 4), (2109, 4), (2109, 4), (543, 0), (24, 0), (254, 3), (2109, 0), (11, 0), (331, 0), (538, 0), (82, 0), (4, 0), (224, 2), (9, 0), (175, 0), (2, 0), (348, 1), (1, 0), (2111, 0)]
[(2110, 0), (1507, 0), (609, 0), (39, 0), (2109, 0), (199, 0), (46, 0), (633, 0), (7, 0), (52, 0), (1467, 0), (5, 0), (210

In [56]:
train_Data = [list(zip(*sent)) for sent in train_corpus]
val_Data = [list(zip(*sent)) for sent in val_corpus]

### Paso 4. Selección de modelo y entrenamiento

Usaré un modelo de red neuronal recurrente con una capa de embedding, una capa oculta de recurrencia, y una capa de salida.

La capa de embedding se define como:

$$ x = Cs $$

$C$ es una matriz de dimensión $d\times N_{in}$ y $s$ es la representación one hot de la palabra en la secuencia.

La capa oculta recurrente:

$$ h^{(t)} = \tanh{(Vh^{(t-1)}+Ux^{(t)})} $$

$V$ es una matriz de dimensión $m\times m$, $U$ una matriz de dimensión $d\times m $.

Y la capa de salida:

$$ y^{(t)} = Softmax(Wh^{(t-1)}+c) $$

En donde $W$ es una matriz de dimensión $N_{out} \times m$

Los valores de $d$ y $m$ son hiperparámetros

#### Función de Riesgo

La función de riesgo será la entroía cruzada:

$$ R(y, \hat{y}) = -\frac1N \sum_{i=0}^Ny\log\hat{y_i} $$

#### Defino los hiperparámetros

In [16]:
dim_in = len(index_to_word)
dim_out = len(tag_to_index)

dim_m = 5
dim_d = 10

print(f'Dimensión de entrada: {dim_in}')
print(f'Dimensión de salida: {dim_out}')
print(f'Dimensión de capa de embedding: {dim_d}')
print(f'Dimensión de capa oculta: {dim_m}')

Dimensión de entrada: 2112
Dimensión de salida: 17
Dimensión de capa de embedding: 10
Dimensión de capa oculta: 5


#### Defino el modelo

In [58]:
class Modelo(object):
    def __init__(self, d, m, N_in, N_out):
        self.d = d # DImensión de capa de embedding
        self.m = m # Dimensión de capa oculta
        self.N_in = N_in # Dimensión de entrada
        self.N_out = N_out # Dimensión de salida

    def inicializar_pesos(self):
        # Capa de embedding
        self.C = np.random.randn(self.d, self.N_in) / np.sqrt(self.N_in)
                
        # Capa recurrente oculta
        self.V = np.random.randn(self.m, self.m) / np.sqrt(self.m)
        self.U = np.random.randn(self.m, self.d) / np.sqrt(self.d)

        # Capa de salida
        self.W = np.random.randn(self.N_out, self.m) / np.sqrt(self.m) 
        self.c = np.zeros(self.N_out)
        
        # Los mejores pesos del modelo
        self.best_C = self.C        
        self.best_V = self.W
        self.best_U = self.U
        self.best_W = self.W
        self.best_c = self.c
    
    def forward(self, secuencia):
        """Paso hacia adelante de la red. 
        Recibe como argumento una secuencia de entrada de n elementos, estos elementos
        deben ser indices del vocabulario de entrada.
        Da como salida $n$ vectores $h$, y $n$ vectores $y$
        """
        # Tamaño de la secuencia de entrada
        T = len(secuencia)

        # Prealojamiento de salidas por estado de capas oculta y de salida.
        h = np.zeros((T+1, self.m)) # Uso un estado adicional para t = -1
        # Se inicia el vector h[-1] con ceros
        h[-1] = np.zeros(self.m)

        y = np.zeros((T, self.N_out))
        x_emb = np.zeros((T, self.d))


        # Propagación hacia adelante de la secuenca de entrada
        for t in range(T):
            # Un paso en la capa embedding se reduce a C[:, s]
            x_emb[t] = self.C[:, secuencia[t]]

            # Capa oculta recurrente
            Vh = self.V.dot(h[t-1])
            Ux = self.U.dot(x_emb[t])
            h[t] = np.tanh(Vh + Ux) # Salida de capa oculta

            # Capa de salida
            a = self.W.dot(h[t]) + self.c # Preactivación
            exp_a = np.exp(a - a.max()) # Exponencial de la preactivación
            y[t] = exp_a/exp_a.sum() # Salida Softmax        
        return y, h, x_emb
    
    def predecir(self, secuencia):
        y_hat, _, _ = self.forward(secuencia)
        return np.argmax(y_hat, axis=1)

    def calcular_riesgo_total(self, x, y):
        """Calculamos la función de riesgo definida por:
        $$ R(y, \hat{y}) = -\frac1N \sum_{n\in N_{in}}y\log\hat{y_n} $$
        x es una lista de secuencias de entrada
        y es una lista de secuencias con los valores reales de 
        """
        R = 0
        N = len(y)
        for i in np.arange(N):
            output, _, _ = self.forward(x[i])
            # La probabilidad de la "y" verdadera
            y_hat_n = output[np.arange(len(y[i])), y[i]]
            R -= np.sum(np.log(y_hat_n))
        return R

    def cross_entropy(self, x, y):
        """Calcula el resgo total y lo divide entre el número de ejemplos
        x es una lista de secuencias de entrada
        y es una lista de secuencias con los valores reales de y
        """
        N = np.sum(( len(y_i) for y_i in y ))
        return self.calcular_riesgo_total(x,y) / N

    def backprop(self, x, y, fw_out):
        """Algoritmo de backpropagation. 
        x es una secuencia de T elementos. 
        y es una secuencia de T elementos. 
        fw_out es una tupla de tres elementros (out, h, x_emb). Contiene
        la salida de la red, la salida de la capa oculta y la salida de la
        capa de embedding.
        """
        T = len(y) # Número de entidades en la secuencia
        # output, h, x_emb = self.forward(x)
        output, h, x_emb = fw_out
        
        # Derivadas respecto a las matrices U, V, W, C y el vector c
        dLdU = np.zeros_like(self.U)
        dLdV = np.zeros_like(self.V)

        dLdW = np.zeros_like(self.W)
        dLdc = np.zeros_like(self.c)
        
        dLdC = np.zeros_like(self.C)

        # Backprop
        # Copio el arreglo para no modificar pesos de la salida original
        d_out = np.array(output, copy=True) 
        # Modifico todos los arreglos en el tiempo
        # En cada indice contenido en y, le resto 1
        d_out[np.arange(len(y)), y] -= 1  # p(Tag_k | w_i) - y_k
        
        # Backprop a través del tiempo
        for t in np.arange(T-1,-1,-1): # Desde T hasta 0
            # Derivada de la capa de salida
            dLdW += np.outer(d_out[t], h[t].T)
            dLdc += d_out[t]

            # Variable de celda recurrente
            d_h = self.W.T.dot(d_out[t])*(1-h[t]**2)

            # Variable de la capa de embedding
            d_emb = self.U.T.dot(d_h)            
            
            # Truncamos la propagación a 4 palabras anteriores a la actual
            for step in np.arange(max(0, t-4), t+1)[::-1]:
                # Derivadas de capa recurrente
                dLdV += np.outer(d_h, h[step-1])
                dLdU += np.outer(d_h, x_emb[step])

                dLdC[:, x[step]] += d_emb

                # Actualizamos variable de celda recurrente
                d_h = self.V.T.dot(d_h) * (1-h[step-1]**2)

                # Actualizamos variable de embedding
                d_emb = self.U.T.dot(d_h)

        return dLdC, dLdU, dLdV, dLdW, dLdc

    def gradiente_descendente(self, gradientes, lr=0.01):
        """Método que da un paso en el gradiente descendiente, actualizando los pesos.
        Recibe una tupla con los valores de los gradientes  (dLdC, dLdU, dLdV, dLdW, dLdc)
        """
        # Desenvolver gradientes
        dLdC, dLdU, dLdV, dLdW, dLdc = gradientes
        
        # self.U -= learning_rate * dLdU
        # self.V -= learning_rate * dLdV
        # self.W -= learning_rate * dLdW
        
        # Actualizamos los pesos
        # Capa de salida
        self.W -= lr*dLdW
        self.c -= lr*dLdc
        # Capa recurrente oculta
        self.V -= lr*dLdV
        self.U -= lr*dLdU
        # Capa de embredding
        self.C -= lr*dLdC # Las demás filas no nos interesan, porque son 0

    def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
        # Calculate the gradients using backpropagation. We want to checker if these are correct.
        fw_out = self.forward(x)
        bptt_gradients = self.backprop(x, y, fw_out)
        # List of all parameters we want to check.
        model_parameters = ['C', 'U', 'V', 'W', 'c']
        # Gradient check for each parameter
        for pidx, pname in enumerate(model_parameters):
            test_passed = True
            # Get the actual parameter value from the mode, e.g. model.W
            parameter = operator.attrgetter(pname)(self)
            print("Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape)))
            # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
            it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index
                # Save the original value so we can reset it later
                original_value = parameter[ix]
                # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
                parameter[ix] = original_value + h
                gradplus = self.calcular_riesgo_total([x],[y])
                parameter[ix] = original_value - h
                gradminus = self.calcular_riesgo_total([x],[y])
                estimated_gradient = (gradplus - gradminus)/(2*h)
                # Reset parameter to original value
                parameter[ix] = original_value
                # The gradient for this parameter calculated using backpropagation
                backprop_gradient = bptt_gradients[pidx][ix]
                # calculate The relative error: (|x - y|/(|x| + |y|))
                relative_error = np.abs(backprop_gradient - estimated_gradient) / \
                                (np.abs(backprop_gradient) + np.abs(estimated_gradient))
                # If the error is to large fail the gradient check
                if relative_error > error_threshold:
                    test_passed = False
                    print("Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix))
                    print("+h Loss: %f" % gradplus)
                    print("-h Loss: %f" % gradminus)
                    print("Estimated_gradient: %f" % estimated_gradient)
                    print("Backpropagation gradient: %f" % backprop_gradient)
                    print("Relative Error: %f" % relative_error)
                    print()
                    break 
                it.iternext()
            if test_passed:
                print("Gradient check for parameter %s passed." % (pname))
                print()
        
    def entrenar(self, train_Data, val_Data, epochs=50, lr=0.1):
        # Historial para guardar las métricas
        history = {
            'train' : {
                'loss' : np.zeros(epochs),
                'conf_mat' : np.zeros((epochs, self.N_out, self.N_out)),
            },
            'val' : {
                'loss' : np.zeros(epochs),
                'conf_mat' : np.zeros((epochs, self.N_out, self.N_out)),
            }
        }
        min_train_loss = np.inf
        min_val_loss = np.inf

        for epoch in tqdm(np.arange(epochs)):
            np.random.shuffle(train_Data)

            # TRAIN
            train_loss = 0
            train_len = len(train_Data)
            train_conf_mat = np.zeros((self.N_out, self.N_out))
            for x, y in train_Data:
                # FORWARD
                len_seq = len(y)
                fw_out = self.forward(x)

                y_hat = fw_out[0]
                # La probabilidad de la "y" verdadera
                y_hat_n = y_hat[np.arange(len(y)), y]
                # RIESGO
                train_loss -= np.sum(np.log(y_hat_n))/len_seq

                # Matriz de confusión
                y_pred = np.argmax(y_hat, axis=1)
                train_conf_mat[y, y_pred] += 1 # Sumo uno en las intersecciones
                
                # BACKPROP
                grads = self.backprop(x, y, fw_out)
                self.gradiente_descendente(grads, lr)
            train_loss /= train_len

            history['train']['conf_mat']['epoch'] = train_conf_mat
            history['train']['loss']['epoch'] = train_loss

            # VALIDATION
            # for x, y in val_Data:
            #     pass

            if epoch % 5 == 0:
                X_train, Y_train = zip(*train_Data)
                tqdm.write(f'Epoch {epoch+1:3}.\ttrain_loss: {train_loss:2.6f}\tval_loss: {val_loss:2.6f}')
            # if cross_entropy < min_entr:
            #     min_entr = cross_entropy  # ponemos la actual
            #     # y movemos los mejores pesos
            #     self.best_C = self.C
            #     self.best_W = self.W
            #     self.best_U = self.U
            #     self.best_c = self.c
                
            # entr_timeline.append(cross_entropy)    
            # tqdm.write(f'Epoch: {epoch+1} \tEntropía cruzada: {cross_entropy}')
        return history
    
    def cargar_mejores_pesos(self):
        self.C = self.best_C
        self.U = self.best_U
        self.V = self.best_V
        self.W = self.best_W
        self.c = self.best_c
    
    def guardar_pesos(self, archivo):
        """Guarda los pesos del modelo en formato .npz
        """
        try:
            np.savez(
                archivo, 
                C = self.C, 
                U = self.U, 
                b = self.V, 
                W = self.W, 
                c = self.c
            )
            print(f'Archivo {archivo} guardado satisfactoriamente')
            return True
        except Exception as e:
            print('Ocurrió un error al guardar el archivo')
            print(e)
            return False
    
    def cargar_pesos(self, archivo):
        """Carga los pesos del modelo guardados en un archivo formato .npz
        """
        try:
            pesos = np.load(archivo)
            self.C = pesos['C']
            self.U = pesos['U']
            self.b = pesos['V']
            self.W = pesos['W']
            self.c = pesos['c']
            print(f'Pesos desde {archivo} cargados correctamente')
            return True
        except Exception as e:
            print('Ocurrió un error al guardar el archivo')
            print(e)
            return False
        

In [29]:
# import operator
modelo_test = Modelo(N_in = 35, d = 20, m = 10, N_out = 15)
modelo_test.inicializar_pesos()
modelo_test.gradient_check([0,1,8,3], [1,2,7,10], h=0.001, error_threshold=0.01)

Performing gradient check for parameter C with size 700.
Gradient check for parameter C passed.

Performing gradient check for parameter U with size 200.
Gradient check for parameter U passed.

Performing gradient check for parameter V with size 100.
Gradient check for parameter V passed.

Performing gradient check for parameter W with size 150.
Gradient check for parameter W passed.

Performing gradient check for parameter c with size 15.
Gradient check for parameter c passed.



#### Instancio el modelo

In [54]:
modelo = Modelo(d = dim_d, m=dim_m, N_in = dim_in, N_out = dim_out)
modelo.inicializar_pesos()

In [57]:
%%time
history = modelo.entrenar(train_Data, [], epochs=20, lr=0.1)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [ ]:
# plt.plot()

In [24]:
# [list(zip(*sent)) for sent in train_corpus][0][0]

(2110,
 123,
 13,
 0,
 692,
 341,
 4,
 178,
 1599,
 636,
 1211,
 11,
 326,
 480,
 5,
 2109,
 129,
 2,
 171,
 1670,
 1,
 2111)

In [43]:
# ñ = np.zeros((5,5))

In [46]:
# ñ[[1,2,3],[0,3,2]]=1

In [47]:
# ñ

array([[0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0.]])


5. Evaluar el desempeño del sistema a partir del corpus de evaluación y con la métrica de Exactitud (Accuracy).
6. Ejemplificar el reconocimiento de entidades nombradas con 5 sentencias del corpus de evaluación.